In [1]:
import time
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from numba import jit
steps_till_convergence = 10
n = 13
reduce = 2

In [2]:
def compute_indices(n, i, j):
    indices_21 = np.ravel_multi_index((i+2, j+1), (n+2,n+2), order='C')
    indices_01 = np.ravel_multi_index((i, j+1), (n+2,n+2), order='C')
    indices_12 = np.ravel_multi_index((i+1, j+2), (n+2,n+2), order='C')
    indices_10 = np.ravel_multi_index((i+1, j), (n+2,n+2), order='C')
    indices_11 = np.ravel_multi_index((i+1, j+1), (n+2,n+2), order='C')
    indices_00s = np.ravel_multi_index((i, j), (n,n), order='C')
    return indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s

@jit(nopython=True)
def compute_inverse_numba(Gx, Gy):
    Gx2 = np.sum(Gx**2)
    Gy2 = np.sum(Gy**2)
    GxGy = np.sum(Gx * Gy)
    A_inv = 1/(GxGy**2 - Gx2*Gy2) * np.array([[GxGy, -Gx2], [-Gy2, GxGy]])
    return A_inv

def compute_inverse_numba2(Gx, Gy):
    Gx2 = np.sum(Gx**2)
    Gy2 = np.sum(Gy**2)
    GxGy = np.sum(Gx * Gy)
    A_inv = 1/(GxGy**2 - Gx2*Gy2) * np.array([[GxGy, -Gx2], [-Gy2, GxGy]])
    return A_inv

@jit(nopython=True)
def compute_delta_numba(F, G, Gx, Gy, A_inv):
    F_G = G - F
    b = np.array([np.sum(Gx*F_G), np.sum(Gy*F_G)])
    delta = np.dot(A_inv, b)
    error = np.sqrt(np.sum(delta**2))
    return delta, error

def compute_delta_numba2(F, G, Gx, Gy, A_inv):
    F_G = G - F
    b = np.array([np.sum(Gx*F_G), np.sum(Gy*F_G)])
    delta = np.dot(A_inv, b)
    error = np.sqrt(np.sum(delta**2))
    return delta, error

# ORIGNAL IMPLEMENTATION
def original_implementation(A, F):
    # Compute the gradients
    im1 = A[2:, 1:-1]
    im2 = A[:-2, 1:-1]
    Gy = (im1 - im2)/2

    im1 = A[1:-1, 2:]
    im2 = A[1:-1, :-2]
    Gx = (im1 - im2)/2

    # Compute the inverse of the matrix
    A_inv = compute_inverse_numba2(Gx, Gy)
    
    # Optimize displacment
    for i in range(steps_till_convergence):
        delta, error = compute_delta_numba2(F, A[1:-1, 1:-1], Gx, Gy, A_inv)
    return Gy

# ORIGNAL IMPLEMENTATION with numba
@jit(nopython=True)
def original_implementation_numba(A, F):
    # Compute the gradients
    im1 = A[2:, 1:-1]
    im2 = A[:-2, 1:-1]
    Gy = (im1 - im2)/2

    im1 = A[1:-1, 2:]
    im2 = A[1:-1, :-2]
    Gx = (im1 - im2)/2

    # Compute the inverse of the matrix
    A_inv = compute_inverse_numba(Gx, Gy)
    
    # Optimize displacment
    for i in range(steps_till_convergence):
        delta, error = compute_delta_numba(F, A[1:-1, 1:-1], Gx, Gy, A_inv)
    return Gy

# ORIGNAL IMPLEMENTATION flat indexing
# Same as above, but with flat indexing instead of slicing. Just in order to compare with the custom windowed implementation
@jit(nopython=True) # This is not possible with flat indexing
def original_implementation_flat_indexing(A_flat, F_flat, indices_21, indices_01, indices_12, indices_10, indices_11):
    im1 = A_flat[indices_21]
    im2 = A_flat[indices_01]
    Gy = (im1 - im2)/2
    im1 = A_flat[indices_12]
    im2 = A_flat[indices_10]
    Gx = (im1 - im2)/2
    A_inv = compute_inverse_numba(Gx, Gy)
    A_clipped = A_flat[indices_11]
    for i in range(steps_till_convergence):
        delta, error = compute_delta_numba(F_flat, A_clipped, Gx, Gy, A_inv)
    return Gy

# Custom Windowed Implementation
@jit(nopython=True)
def custom_windowed_implementation(A_flat, F_flat, indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s):
    im1 = A_flat[indices_21]
    im2 = A_flat[indices_01]
    Gy = (im1 - im2) / 2
    im1 = A_flat[indices_12]
    im2 = A_flat[indices_10]
    Gx = (im1 - im2)/2
    A_inv = compute_inverse_numba(Gx, Gy)
    A_clipped_new = A_flat[indices_11]
    F_clipped_new = F_flat[indices_00s]
    for i in range(steps_till_convergence+1):
        delta, error = compute_delta_numba(F_clipped_new, A_clipped_new, Gx, Gy, A_inv)
    return Gy

# # Custom Windowed Implementation
# @jit(nopython=True)
# def custom_windowed_implementation(A, F, include):
#     im1 = A[2:, 1:-1].ravel()[include.ravel()]
#     im2 = A[:-2, 1:-1].ravel()[include.ravel()]
#     Gy = (im1 - im2) / 2
#     im1 = A[1:-1, 2:].ravel()[include.ravel()]
#     im2 = A[1:-1, :-2].ravel()[include.ravel()]
#     Gx = (im1 - im2)/2
#     A_inv = compute_inverse_numba(Gx, Gy)
#     A_clipped_new = A[1:-1, 1:-1].ravel()[include.ravel()]
#     F_clipped_new = F.ravel()[include.ravel()]
#     for i in range(steps_till_convergence+1):
#         delta, error = compute_delta_numba(F_clipped_new, A_clipped_new, Gx, Gy, A_inv)
#     return Gy

# Custom Windowed Implementation flat
def custom_windowed_implementation_flat(A, F, indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s):
    im1 = A.flat[indices_21]
    im2 = A.flat[indices_01]
    Gy = (im1 - im2)/2
    im1 = A.flat[indices_12]
    im2 = A.flat[indices_10]
    Gx = (im1 - im2)/2
    A_inv = compute_inverse_numba(Gx, Gy)
    A_clipped_new = A.flat[indices_11]
    F_clipped_new = F.flat[indices_00s]
    for i in range(steps_till_convergence+1):
        delta, error = compute_delta_numba(F_clipped_new, A_clipped_new, Gx, Gy, A_inv)
    return Gy

def parallel_implementation(A, F, Gy, Gx):
    # Compute the gradients
    im1 = A[2:, 4:7]
    im2 = A[:-2, 4:7]
    Gy = (im1 - im2)/2

    im1 = A[4:7, 2:]
    im2 = A[4:7, :-2]
    Gx = (im1 - im2)/2
    # Compute the inverse of the matrix
    A_inv = compute_inverse_numba(Gx, Gy)
    im1 = A[6:9, 1:-1]
    im2 = A[4:7, 1:-1]
    Gy = (im1 - im2)/2
    im1 = A[1:-1, 6:9]
    im2 = A[1:-1, 4:7]
    Gx = (im1 - im2)/2
    A_inv += compute_inverse_numba(Gx, Gy)

    # Optimize displacment
    for i in range(steps_till_convergence):
        delta, error = compute_delta_numba(F, A[1:-1, 1:-1], Gx, Gy, A_inv)
    return Gy



In [3]:
n = 12
A = np.arange(n)
A = A[:, np.newaxis] + A[np.newaxis, :]
A, (A[2:, 4:7] - A[:-2, 4:7])/2, (A[1:-1, 2:] - A[1:-1, :-2])/2

(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
        [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
        [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
        [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]]),
 array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]),
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1

In [4]:
n = 13
reduce = 2

time_full_total = 0
time_full_numba_total = 0
time_full_flat_total = 0
time_partial_total = 0
time_partial_flat_total = 0
total_steps = 10000

include_all = True * np.ones((n,n), dtype=bool)
i, j = np.where(include_all)
indices_21_all, indices_01_all, indices_12_all, indices_10_all, indices_11_all, indices_00s_all = compute_indices(n, i, j)

for it in range(total_steps):
    # Generate two frames of random data
    A = np.random.randint(0, 255, (n+2, n+2))
    F = np.random.randint(0, 255, (n, n))
    # Randomly select pixels to exclude from analysis
    include = True * np.ones((n,n), dtype=bool)
    num_elements = n**2
    num_false = n**2 - (n-reduce)**2
    # Find the corresponding indices
    indices = np.random.choice(num_elements, num_false, replace=False)
    include.flat[indices] = False
    i, j = np.where(include)
    # Set up flat indexing. Each frame the same indexing is used so they are precomputed
    indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s = compute_indices(n, i, j)

    ### Compare the different L-K implementations
    # ORIGNAL IMPLEMENTATION
    start_time = time.time()
    Gy_ori = original_implementation(A, F)
    end_time = time.time()
    time_full = end_time - start_time
    time_full_total += time_full

    # ORIGNAL IMPLEMENTATION with numba
    start_time = time.time()
    Gy_numba_ori = original_implementation_numba(A, F)
    end_time = time.time()
    time_numba_full = end_time - start_time
    time_full_numba_total += time_numba_full

    # # ORIGNAL IMPLEMENTATION flat indexing
    A_flat = A.flatten()
    start_time = time.time()
    Gy_fla = original_implementation_flat_indexing(A_flat, F.flatten(), indices_21_all, indices_01_all, indices_12_all, indices_10_all, indices_11_all)
    end_time = time.time()
    time_full_flat = end_time - start_time
    time_full_flat_total += time_full_flat

    # NEW IMPLEMENTATION
    start_time = time.time()
    Gy_new = custom_windowed_implementation(A_flat, F.flatten(), indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s)
    end_time = time.time()
    time_partial = end_time - start_time
    time_partial_total += time_partial

    # # NEW IMPLEMENTATION flat indexing
    start_time = time.time()
    Gy_new_fla = custom_windowed_implementation_flat(A, F, indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s)
    end_time = time.time()
    time_partial_flat = end_time - start_time
    time_partial_flat_total += time_partial_flat

result = np.array_equal(Gy_ori[include], Gy_fla[indices_00s])
result2 = np.array_equal(Gy_ori[include], Gy_new)
result3 = np.array_equal(Gy_ori[include], Gy_new_fla)

table = [["Comparison", "Result"],
        ["Gy(full slicing) == Gy(full flat indexing)", result],
        ["Gy(full slicing) == Gy(partial flat indexing)", result2],
        ["Gy(full slicing) == Gy(partial flat indexing)", result3],
        ["Time (full window with slicing)", time_full_total/total_steps],
        ["Time (full  window with flat indexing)", time_full_flat_total/total_steps],
        ["Time (partial numba)", time_partial_total/total_steps],
        ["Time (partial flat indexing)", time_partial_flat_total/total_steps],
        ["performance gain % (full slicing vs. partial flat indexing)", (time_full_total-time_partial_total)/time_full_total*100],
        ["performance gain % (full flat indexing vs. partial flat indexing)", (time_full_flat_total-time_partial_total)/time_full_flat_total*100]]

table_str = tabulate(table, headers="firstrow", tablefmt="fancy_grid")
print(table_str)


╒═══════════════════════════════════════════════════════════════════╤══════════════╕
│ Comparison                                                        │       Result │
╞═══════════════════════════════════════════════════════════════════╪══════════════╡
│ Gy(full slicing) == Gy(full flat indexing)                        │  1           │
├───────────────────────────────────────────────────────────────────┼──────────────┤
│ Gy(full slicing) == Gy(partial flat indexing)                     │  1           │
├───────────────────────────────────────────────────────────────────┼──────────────┤
│ Gy(full slicing) == Gy(partial flat indexing)                     │  1           │
├───────────────────────────────────────────────────────────────────┼──────────────┤
│ Time (full window with slicing)                                   │  0.00027672  │
├───────────────────────────────────────────────────────────────────┼──────────────┤
│ Time (full  window with flat indexing)                         

In [5]:
# def test_performance(function, steps):
#     A = np.random.randint(0, 255, (steps,n+2, n+2))
#     F = np.random.randint(0, 255, (steps, n, n))
#     # Randomly select pixels to exclude from analysis
#     include = True * np.ones((n,n), dtype=bool)
#     num_elements = n**2
#     num_false = n**2 - (n-reduce)**2
#     # Find the corresponding indices
#     indices = np.random.choice(num_elements, num_false, replace=False)
#     include.flat[indices] = False
#     i, j = np.where(include)
#     # Set up flat indexing. Each frame the same indexing is used so they are precomputed
#     indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s = compute_indices(n, i, j)
#     time_start = time.time()
#     for i in range(steps):
#         function(A[i], F[i], indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s)
#     time_end = time.time()
#     return time_end - time_start

LL = 0.15
UL = 0.85
n_vec = np.arange(5, 42, 2)
reduce_vec = np.arange(1, 9, 2)
total_steps = 10000
computation_time_dict = {}
for n in n_vec:
    include_all = True * np.ones((n,n), dtype=bool)
    i, j = np.where(include_all)
    indices_21_all, indices_01_all, indices_12_all, indices_10_all, indices_11_all, indices_00s_all = compute_indices(n, i, j)
    for reduce in reduce_vec:
        if n-reduce < 2:
            continue
        time_full_total = []
        time_numba_full_total = []
        time_full_flat_total = []
        time_partial_total = []
        time_partial_flat_total = []
        for it in range(total_steps):
            # Generate two frames of random data
            A = np.random.randint(0, 255, (n+2, n+2))
            F = np.random.randint(0, 255, (n, n))
            # Randomly select pixels to exclude from analysis
            include = True * np.ones((n,n), dtype=bool)
            num_elements = n**2
            num_false = n**2 - (n-reduce)**2
            # Find the corresponding indices
            indices = np.random.choice(num_elements, num_false, replace=False)
            include.flat[indices] = False
            i, j = np.where(include)
            # Set up flat indexing. Each frame the same indexing is used so they are precomputed
            indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s = compute_indices(n, i, j)

            ### Compare the different L-K implementations
            if reduce == 1:
                # ORIGNAL IMPLEMENTATION FULL
                start_time = time.time()
                Gy_ori = original_implementation(A, F)
                end_time = time.time()
                time_full = end_time - start_time
                
                # ORIGNAL IMPLEMENTATION with numba
                start_time = time.time()
                Gy_numba_ori = original_implementation_numba(A, F)
                end_time = time.time()
                time_numba_full = end_time - start_time

                # # ORIGNAL IMPLEMENTATION flat indexing
                A_flat = A.flatten()
                start_time = time.time()
                Gy_fla = original_implementation_flat_indexing(A_flat, F.flatten(), indices_21_all, indices_01_all, indices_12_all, indices_10_all, indices_11_all)
                end_time = time.time()
                time_full_flat = end_time - start_time

            # NEW IMPLEMENTATION
            A_flat = A.flatten()
            start_time = time.time()
            Gy_new = custom_windowed_implementation(A_flat, F.flatten(), indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s)
            end_time = time.time()
            time_partial = end_time - start_time

            # # NEW IMPLEMENTATION flat indexing
            start_time = time.time()
            Gy_new_fla = custom_windowed_implementation_flat(A, F, indices_21, indices_01, indices_12, indices_10, indices_11, indices_00s)
            end_time = time.time()
            time_partial_flat = end_time - start_time

            if reduce == 1:
                time_full_total.append(time_full)
                time_numba_full_total.append(time_numba_full)
                time_full_flat_total.append(time_full_flat)
            time_partial_total.append(time_partial)
            time_partial_flat_total.append(time_partial_flat)
        # Convert the lists to numpy arrays
        time_full_total = np.array(time_full_total)
        time_numba_full_total = np.array(time_numba_full_total)
        time_full_flat_total = np.array(time_full_flat_total)
        time_partial_total = np.array(time_partial_total)
        time_partial_flat_total = np.array(time_partial_flat_total)

        # Compute the IQR and remove outliers
        if reduce == 1:
            time_full_total = np.mean(time_full_total)
            time_numba_full_total = np.mean(time_numba_full_total)
            time_full_flat_total = np.mean(time_full_flat_total)
        time_partial_total = np.mean(time_partial_total)
        time_partial_flat_total = np.mean(time_partial_flat_total)

        computation_time_dict[(n, reduce)] = [time_full_total/total_steps, time_numba_full_total/total_steps, time_full_flat_total/total_steps, time_partial_total/total_steps, time_partial_flat_total/total_steps]
        print(f"n: {n}, reduce: {reduce}, time_full_total: {time_full_total/total_steps}, time_full_flat_total: {time_full_flat_total/total_steps}, time_partial_total: {time_partial_total/total_steps}, time_partial_flat_total: {time_partial_flat_total/total_steps}")


n: 5, reduce: 1, time_full_total: 2.6974556446075438e-08, time_full_flat_total: 1.1805367469787598e-09, time_partial_total: 1.096343994140625e-09, time_partial_flat_total: 3.920693397521973e-09
n: 5, reduce: 3, time_full_total: [], time_full_flat_total: [], time_partial_total: 1.2705421447753907e-09, time_partial_flat_total: 3.726174831390381e-09
n: 7, reduce: 1, time_full_total: 2.7381856441497802e-08, time_full_flat_total: 1.0154008865356444e-09, time_partial_total: 9.50791835784912e-10, time_partial_flat_total: 4.415900707244872e-09
n: 7, reduce: 3, time_full_total: [], time_full_flat_total: [], time_partial_total: 1.3637304306030273e-09, time_partial_flat_total: 3.972392082214355e-09
n: 7, reduce: 5, time_full_total: [], time_full_flat_total: [], time_partial_total: 1.2747573852539063e-09, time_partial_flat_total: 3.902339935302734e-09
n: 9, reduce: 1, time_full_total: 2.716960668563843e-08, time_full_flat_total: 1.1156010627746582e-09, time_partial_total: 1.8156003952026365e-09, t

In [6]:
%matplotlib qt
ls = ['x', '*', 'o', 'v', '^', '<', '>']
fig, ax = plt.subplots()

label_added = False

for (n, reduce), value in computation_time_dict.items():
    if reduce == 1:
        ax.plot(n, value[0], '.r', label='Full - Slicing  - no Numba' if not label_added else '')
        ax.plot(n, value[1], '.b', label='Full - Slicing - with Numba' if not label_added else '')
        ax.plot(n, value[2], '.c', label='Full - Flat index- with Numba' if not label_added else '')
    ax.plot(n, value[3], 'g' + ls[(reduce-1) % len(ls)], label='Partial - Flat Index - with Numba {}'.format(reduce) if not label_added else '')
    ax.plot(n, value[4], 'y' + ls[(reduce-1) % len(ls)], label='Partial - Flat Index - no Numba {}'.format(reduce) if not label_added else '')
    label_added = True

ax.set_xlabel('window size n')
ax.set_ylabel('Computation Time of single window of single frame')
ax.legend()



In [7]:
# Create a C-contiguous array of random integers with bit depth 12
A = np.random.randint(0, 2**12, size=(1024, 512), dtype=np.int16)
print(A.flags['C_CONTIGUOUS'])

time_start = time.time()
A_fortran = np.asfortranarray(A)
time_end = time.time()

print(A_fortran.flags['C_CONTIGUOUS'])
print((time_end - time_start)*8000)

True
False
7.77435302734375
